In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input_data/gg_regression_table_for_descriptives.csv')

In [ ]:
df.AverageSimilarity.mean()

In [ ]:
df_cand = df[(df.candidate == 1)]

## DV descriptives

In [ ]:
source_path = '' # Local source path

In [ ]:
var = pd.read_csv(os.path.join(source_path, 'g2022_adid_var_sets_091224.csv.gz'))

In [ ]:
var = var.loc[var.set3 == 1]

**General election period**

In [ ]:
condition = (var.date_range_end >= '2022-09-01') & (var.date_range_start <= '2022-11-30')
'''
Second condition is not necessary: resulting dataframe shapes are the same.
'''
var = var[condition]

In [ ]:
text = pd.read_csv(os.path.join(source_path, 'g2022_adid_01062021_11082022_text_v20240115.csv.gz'))

In [ ]:
var = var.merge(text, how='left', on='ad_id').drop_duplicates()

In [ ]:
cid_table_path = '../../../../google_2022/produce_complete_data/04-dedup-2nd-time/final_output/cid_google2022_v20231203.csv'
cid = pd.read_csv(cid_table_path)

In [ ]:
var = var.merge(cid, how='left', on='ad_id').drop_duplicates()

In [ ]:
del text, cid

In [ ]:
var.columns

**Number of unique creatives: General election period**

In [ ]:
sns.set_theme(style='ticks', palette="tab10")

g = sns.histplot(df.AverageSimilarity, kde=True, bins=40, color='gray', line_kws={'color': 'gray'})

g.set(title='Distribution of average pairwise similarity at the race-sponsor-media level\n\nGoogle', xlabel='Average similarity')
g.title.set_size(10)

In [ ]:
df_agg = df.merge(var[['advertiser_id', 'advertiser_name']], how='left', on='advertiser_id').drop_duplicates()

In [ ]:
df_agg['advertiser_name'] = df_agg['advertiser_name'].str.lower()

In [ ]:
df_agg = df_agg.groupby(['advertiser_id']).agg({'advertiser_name': 'first', 'num_unique': 'sum'}).reset_index() 

In [ ]:
df_agg.num_unique.mean()

In [ ]:
df_agg.num_unique.median()

In [ ]:
df_agg[df_agg.advertiser_name == 'nrsc'] 

In [ ]:
df_agg.sort_values(by='num_unique', ascending=False).head(40)

In [ ]:
# aggregated num_unique histogram (sponsor level)
g = sns.histplot(df_agg.num_unique, kde=True, bins=200, color='gray')
g.set(title='Number of unique creatives at the sponsor level\n\nGoogle', xlabel='')
plt.xlim((0, 100))

### Sophistication index

In [ ]:
def create_sophistication_index(input_df):
    max_ = input_df.num_unique.max()
    min_ = input_df.num_unique.min()
    input_df['num_unique_normalized'] = (input_df.num_unique - min_)/(max_ - min_)
    input_df['sophistication_index'] = input_df['num_unique_normalized'] * (1 - input_df.AverageSimilarity)
    return input_df

In [ ]:
df_cand = create_sophistication_index(df_cand)

In [ ]:
cols = ['advertiser_id', 'advertiser_name', ]
df_cand = df_cand.merge(var[cols], how='left', on='advertiser_id').drop_duplicates()

In [ ]:
df_cand.columns

#### General eleciton period

**To display in paper**

In [ ]:
from IPython.display import display, HTML

In [ ]:
df_cand["advertiser_name"] = df_cand.advertiser_name.str.title()

In [ ]:
df_cand["ad_type"] = df_cand.ad_type.str.title()

**general election period**

In [ ]:
cols2display = ['advertiser_name', 'sophistication_index', 'ad_type', 'num_unique', 'AverageSimilarity']

display_df = df_cand[cols2display].drop_duplicates().sort_values(by='sophistication_index', ascending=False).head(22)

display(HTML(display_df.to_html(index=False)))

In [ ]:
df_cand[cols2display].sort_values(by='sophistication_index', ascending=False).head(50)